# Modelo de red neuronal

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Cargar el modelo y el tokenizer de RoBERTuito entrenado para análisis de sentimiento
modelo = "pysentimiento/robertuito-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(modelo)
model = AutoModelForSequenceClassification.from_pretrained(modelo)

# Crear pipeline para clasificación
analizador = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

c:\Users\nerea\anaconda3\envs\sentimentanal\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


In [3]:
# Frase de ejemplo
frase1 = "Estoy harto de todo esto"
frase2 = "Me encanta la programación en Python"
frase3 = "Hoy hace sol"

# Clasificar
resultado = analizador(frase1)
print(resultado)
resultado = analizador(frase2)
print(resultado)
resultado = analizador(frase3)
print(resultado)

[{'label': 'NEG', 'score': 0.9645105600357056}]
[{'label': 'POS', 'score': 0.9567092061042786}]
[{'label': 'NEU', 'score': 0.560903787612915}]


Como se puede apreciar, el modelo de red neuronal da una salida del tipo:
- (sentimiento, score)
donde sentimiento puede ser NEG (negativo), NEU (neutral) o POS (positivo) y score es un número entre 0 y 1, indicando una puntuación asignada por el modelo al sentimiento inferido.

Por tanto, no se trata de un modelo de regresión, o de clasificación binaria, sino de un modelo de clasificación multiclase, donde la salida es la tupla (sentimiento, score).

De cara a poder agregar valores de score, es necesario transformar de acuerdo a nuestro interés. No podemos simplemente sumar los scores, ya que el score por si solo no indica el sentimiento de la frase. En nuestro caso queremos diferenciar entre sentimiento (positivo o negativo) y neutralidad. Por tanto, tiene sentido utilizar el score como un peso y asignar un valor numérico a las etiquetas:

- NEG = 1
- NEU = -1
- POS = 1

Como solo queremos diferenciar entre sentimiento y no sentimiento (para la hipótesis 3), asignamos el valor 1 a NEG y POS, y -1 a NEU. De esta forma, si la frase es positiva o negativa, el score se suma al resultado final, mientras que si es neutral, se resta.

Cada frase quedaría entonces caracterizada por la tupla (sentimiento, score) y una puntuación calculada como:
$$
\text{puntuacion} = \text{sentimiento} \cdot \text{score} 
$$
siendo sentimiento el valor numérico asignado a la etiqueta (-1 o 1).

# Prueba del modelo con datos ficticios

In [2]:
# Generamos un conjunto de datos de ejemplo
import pandas as pd
import numpy as np

lider_a_partido={}
data = pd.read_csv("base_conjunta.csv")
# data["Partido"]


In [3]:
lider_a_partido ={'Santiago Abascal':"VOX", 'Feijóo':"PP", 'Pedro Sánchez':"PSOE", 'PP':"PP", 'PSOE':"PSOE",
       'Sumar':"Sumar", 'Yolanda Díaz':"Sumar", 'VOX':'VOX'}
oposicion = set(['PP', 'VOX', 'Sumar'])
data["Partido"] = data["nombre"].map(lider_a_partido)
data["Oposición"] = data["Partido"].apply(lambda x: x in oposicion)

In [4]:
data.head()

,nombre,id,timestamp,Lider,body,blackout,origen,Partido,Oposición
0,Santiago Abascal,1927846794114859511,2025-05-28 23:57:48,si,Llevo tiempo diciendo que lo peor de Sánchez e...,0,ABASCAL,VOX,True
1,Santiago Abascal,1927825174524543304,2025-05-28 22:31:53,si,Será un placer acompañarte.,0,ABASCAL,VOX,True
2,Santiago Abascal,1927718717070217584,2025-05-28 15:28:52,si,Esta es la realidad: el Partido Popular (españ...,0,ABASCAL,VOX,True
3,Santiago Abascal,1927656927925879174,2025-05-28 11:23:20,si,Periodismo lacayo que hace cualquier cosa para...,0,ABASCAL,VOX,True
4,Santiago Abascal,1927430822652752036,2025-05-27 20:24:52,si,El autócrata pretende que su régimen sea perpe...,0,ABASCAL,VOX,True


In [ ]:
from tqdm import tqdm
sentimiento = []
score = []
puntuacion = []
dic = {"NEU":-1, "POS":1, "NEG":1}

for frase in tqdm(data["body"]):
    try:
      resultado = analizador(frase)
    except Exception as e:
      print(f"Error procesando la frase: {frase}")
      print(f"Error: {e}")
  # Guardamos la etiqueta de sentimiento
  sentimiento.append(resultado[0]['label'])
  # guardamos el score
  score.append(resultado[0]['score'])
  
  # calculamos la puntuación multiplicando el score por el valor de la etiqueta
  puntuacion.append(dic[resultado[0]['label']]*resultado[0]['score'])

  
data["Sentimiento"] = sentimiento
data["Score"] = score
data["Puntuacion"] = puntuacion
df = pd.DataFrame(data)
print("Resultados")
print(df)
print("\n")

# Sumamos puntuacion agrupado por partido
print("Resultados por partido (sumando la puntuacion)")
df_partido = df.groupby("Partido")["Puntuacion"].sum().reset_index()
print(df_partido)
print("\n")
# Sumamos puntuacion agrupado por partido y lider
print("Resultados por partido y lider (sumando la puntuacion)")
df_partido_lider = df.groupby(["Partido", "Lider"])["Puntuacion"].sum().reset_index()
print(df_partido_lider)
print("\n")

 77%|███████▋  | 834/1087 [08:52<02:41,  1.57it/s]  


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

Los resultados ficticios nos indican lo siguiente:

- A nivel de partido, todos los valores son positivos, indicando que en general los partidos muestra sentimientos en las frases.
- A nivel de líderes, de nuevo todos los valores son positivos. Destacaríamos que el lider del partido A tiene un score más cercano al 0, lo que indica una carga sentimental más baja que el resto de líderes.

In [ ]:
# Confianza de la oposicion que sea negativo
# Tweets negativos de la oposicion
df_oposicion = df[df["Oposicion"] == True]
df_oposicion_neg = df_oposicion[df_oposicion["Sentimiento"] == "NEG"]


confianza = df_oposicion_neg.shape[0] / df_oposicion.shape[0]
print("Confianza de la oposicion que sea negativo: ", confianza)

# Lift
lift = confianza / (df[df["Sentimiento"] == "NEG"].shape[0] / df.shape[0])
print("Lift: ", lift)

Confianza de la oposicion que sea negativo:  1.0
Lift:  1.2


Hipotesis 3

In [ ]:
data_ps = data[data["nombre"]=="Pedro Sánchez"]
data_nf = data[data["nombre"]=="Feijóo"]
data_ab = data[data["nombre"]=="Santiago Abascal"]
data_yd = data[data["nombre"]=="Yolanda Díaz"]
data_su = data[data["nombre"]=="Sumar"]
data_pp = data[data["nombre"]=="PP"]
data_vox = data[data["nombre"]=="VOX"]
data_psoe = data[data["nombre"]=="PSOE"]


# Aplicar test de normalidad a cada dataset usando {}

# Aplicar test de hipótesis por pares (en Puntuación)
# Usando el modelo {modelo}